In [65]:
# !pip install streamlit
# !pip install pandas
# !pip install numpy
# !pip install groq
# !pip install langchain_community
# !pip install langchain_groq
# !pip install yfinance
# !pip install plotly
# !pip install langchain_core
# !pip install langchain-pinecone

In [66]:
from langchain_groq import ChatGroq
import os
import yfinance as yf
import pandas as pd

from langchain_core.tools import tool
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, ToolMessage

from datetime import date
import pandas as pd
import plotly.graph_objects as go

import streamlit as st

import streamlit as st
import pandas as pd
import numpy as np
from groq import Groq
from pinecone import Pinecone

from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_pinecone import PineconeVectorStore


In [52]:
#define llm (Groq)
llm = ChatGroq(groq_api_key= "", model= "llama3-8b-8192")


In [53]:
@tool
def get_historical_price(symbol, start_date, end_date):
    """
    Fetches historical stock prices for a given symbol from 'start_date' to 'end_date'.
    - symbol (str): Stock ticker symbol.
    - end_date (date): Typically today unless a specific end date is provided. End date MUST be greater than start date
    - start_date (date): Set explicitly, or calculated as 'end_date - date interval' (for example, if prompted 'over the past 6 months', date interval = 6 months so start_date would be 6 months earlier than today's date). Default to '1900-01-01' if vaguely asked for historical price. Start date must always be before the current date
    """
    data = yf.Ticker(symbol)
    hist = data.history(start=start_date, end=end_date)
    return hist
    

In [54]:
print(yf.Ticker("AMZN"))

yfinance.Ticker object <AMZN>


In [55]:
@tool
def get_stock_info(symbol,key):
    '''Return the correct stock info value given the appropriate symbol and key. Infer valid key from the user prompt; it must be one of the following:

    address1, city, state, zip, country, phone, website, industry, industryKey, industryDisp, sector, sectorKey, sectorDisp, longBusinessSummary, fullTimeEmployees, companyOfficers, auditRisk, boardRisk, compensationRisk, shareHolderRightsRisk, overallRisk, governanceEpochDate, compensationAsOfEpochDate, maxAge, priceHint, previousClose, open, dayLow, dayHigh, regularMarketPreviousClose, regularMarketOpen, regularMarketDayLow, regularMarketDayHigh, dividendRate, dividendYield, exDividendDate, beta, trailingPE, forwardPE, volume, regularMarketVolume, averageVolume, averageVolume10days, averageDailyVolume10Day, bid, ask, bidSize, askSize, marketCap, fiftyTwoWeekLow, fiftyTwoWeekHigh, priceToSalesTrailing12Months, fiftyDayAverage, twoHundredDayAverage, currency, enterpriseValue, profitMargins, floatShares, sharesOutstanding, sharesShort, sharesShortPriorMonth, sharesShortPreviousMonthDate, dateShortInterest, sharesPercentSharesOut, heldPercentInsiders, heldPercentInstitutions, shortRatio, shortPercentOfFloat, impliedSharesOutstanding, bookValue, priceToBook, lastFiscalYearEnd, nextFiscalYearEnd, mostRecentQuarter, earningsQuarterlyGrowth, netIncomeToCommon, trailingEps, forwardEps, pegRatio, enterpriseToRevenue, enterpriseToEbitda, 52WeekChange, SandP52WeekChange, lastDividendValue, lastDividendDate, exchange, quoteType, symbol, underlyingSymbol, shortName, longName, firstTradeDateEpochUtc, timeZoneFullName, timeZoneShortName, uuid, messageBoardId, gmtOffSetMilliseconds, currentPrice, targetHighPrice, targetLowPrice, targetMeanPrice, targetMedianPrice, recommendationMean, recommendationKey, numberOfAnalystOpinions, totalCash, totalCashPerShare, ebitda, totalDebt, quickRatio, currentRatio, totalRevenue, debtToEquity, revenuePerShare, returnOnAssets, returnOnEquity, freeCashflow, operatingCashflow, earningsGrowth, revenueGrowth, grossMargins, ebitdaMargins, operatingMargins, financialCurrency, trailingPegRatio
    
    If asked generically for 'stock price', use currentPrice
    '''
    data = yf.Ticker(symbol)
    stock_info = data.info
    return stock_info[key]

In [57]:
tools = [get_stock_info, get_historical_price]

In [58]:
llm_tools = llm.bind_tools(tools)

In [59]:
system_prompt = 'You are a helpful finance assistant that analyzes stocks and stock prices. Today is {today}'.format(today = date.today())
messages = [
    SystemMessage(system_prompt),
    HumanMessage("Can you please help me to get stock price of  AMZN for last 2 days ?")
]
ai_message = llm_tools.invoke(messages)

In [60]:
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_856x', 'function': {'arguments': '{"symbol":"AMZN","start_date":"2024-05-09","end_date":"2024-05-11"}', 'name': 'get_historical_price'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.104, 'completion_tokens': 89, 'prompt_time': 0.407, 'prompt_tokens': 1694, 'queue_time': None, 'total_time': 0.511, 'total_tokens': 1783}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-52863e57-b178-4615-88c9-2f1061ef2749-0', tool_calls=[{'name': 'get_historical_price', 'args': {'symbol': 'AMZN', 'start_date': '2024-05-09', 'end_date': '2024-05-11'}, 'id': 'call_856x'}])

In [61]:
messages.append(ai_message)
historical_price_dfs = []
symbols = []
tool_dict = {"get_stock_info": get_stock_info, "get_historical_price": get_historical_price}

In [62]:
for tool_call in ai_message.tool_calls:
    # print(tool_call)
    selected_tool = tool_dict[tool_call['name']]
    output  = selected_tool.invoke(tool_call['args'])
    print(output)
    

                                 Open        High         Low       Close  \
Date                                                                        
2024-05-09 00:00:00-04:00  188.880005  191.699997  187.440002  189.500000   
2024-05-10 00:00:00-04:00  189.160004  189.889999  186.929993  187.479996   

                             Volume  Dividends  Stock Splits  
Date                                                          
2024-05-09 00:00:00-04:00  43368400        0.0           0.0  
2024-05-10 00:00:00-04:00  34121700        0.0           0.0  
